In [8]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [ ]:


# ----------------------------------
# Function to show images step-by-step
# ----------------------------------
def show_image(img, title="Image"):
    plt.figure(figsize=(8,6))
    if len(img.shape) == 2:
        plt.imshow(img, cmap='gray')
    else:
        plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.title(title)
    plt.axis('off')
    plt.show()

# ----------------------------------
# Step 1: Load Image
# ----------------------------------
image_path = 'test2.jpg'  # 🔁 Change this to your image path
image = cv2.imread(image_path)
original = image.copy()

show_image(image, "Original Image")

# ----------------------------------
# Step 2: Grayscale Conversion
# ----------------------------------
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
show_image(gray, "Grayscale Image")

# ----------------------------------
# Step 3: Gaussian Blur + Canny Edge Detection
# ----------------------------------
blurred = cv2.GaussianBlur(gray, (5, 5), 0)
edges = cv2.Canny(blurred, 50, 150)
show_image(blurred, "Blurred Image")
show_image(edges, "Canny Edge Detection")

# ----------------------------------
# Step 4: Find Contours
# ----------------------------------
contours, _ = cv2.findContours(edges.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
contours = sorted(contours, key=cv2.contourArea, reverse=True)[:10]

# Draw all contours to visualize
contour_image = original.copy()
cv2.drawContours(contour_image, contours, -1, (0, 255, 0), 2)
show_image(contour_image, "All Detected Contours")

# ----------------------------------
# Step 5: Find the 4-Corner Contour (ID Card Shape)
# ----------------------------------
id_card_contour = None

for contour in contours:
    peri = cv2.arcLength(contour, True)
    approx = cv2.approxPolyDP(contour, 0.02 * peri, True)

    if len(approx) == 4:
        id_card_contour = approx
        break

# Visual Check
if id_card_contour is not None:
    id_card_outline = original.copy()
    cv2.drawContours(id_card_outline, [id_card_contour], -1, (255, 0, 0), 3)
    show_image(id_card_outline, "Detected 4-Corner ID Card")
else:
    print("❌ Could not find a 4-point contour!")

# ----------------------------------
# Step 6: Perspective Transformation
# ----------------------------------
def order_points(pts):
    pts = pts.reshape(4, 2)
    rect = np.zeros((4, 2), dtype="float32")
    
    s = pts.sum(axis=1)
    rect[0] = pts[np.argmin(s)]  # top-left
    rect[2] = pts[np.argmax(s)]  # bottom-right
    
    diff = np.diff(pts, axis=1)
    rect[1] = pts[np.argmin(diff)]  # top-right
    rect[3] = pts[np.argmax(diff)]  # bottom-left
    
    return rect

if id_card_contour is not None:
    rect = order_points(id_card_contour)
    (tl, tr, br, bl) = rect

    # Compute dimensions
    widthA = np.linalg.norm(br - bl)
    widthB = np.linalg.norm(tr - tl)
    heightA = np.linalg.norm(tr - br)
    heightB = np.linalg.norm(tl - bl)
    
    maxWidth = int(max(widthA, widthB))
    maxHeight = int(max(heightA, heightB))

    # Destination coordinates for perspective transform
    dst = np.array([
        [0, 0],
        [maxWidth - 1, 0],
        [maxWidth - 1, maxHeight - 1],
        [0, maxHeight - 1]
    ], dtype="float32")

    # Compute transformation matrix and warp
    M = cv2.getPerspectiveTransform(rect, dst)
    warped = cv2.warpPerspective(original, M, (maxWidth, maxHeight))
    
    show_image(warped, "Perspective-Corrected & Cropped ID Card")

    # ----------------------------------
    # Step 7: Resize Final Output
    # ----------------------------------
    final = cv2.resize(warped, (300, 150))
    show_image(final, "Final Resized ID Card (300x150)")

    # Optional: Save
    cv2.imwrite("final_id_card.jpg", final)
else:
    print("❌ Cannot transform perspective. No valid contour.")
